# Scratch Artificial Neural Network

In [1]:
from torch.utils.data import DataLoader
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.dont_write_bytecode = True

# Import libraries
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms


from ArtificialNeuralNetwork import ArtificialNeuralNetwork
from Layer import Layer, OutputLayer
from Function import ActivationFunction, LossFunction
from enums import InitializerType

In [3]:
# Disable __grad based on assistant direction
torch.set_grad_enabled(False)

# Pipeline for testing MNIST dataset

In [4]:
input_size = 784
hidden_layers = 2
hidden_size = 128
output_size = 10
learning_rate = 0.001
param_1 = 0
param_2 = 0

In [5]:
# Import Dataset
train = pd.read_csv("data/train.csv")

In [6]:
 # Define a simple transformation.
transform = transforms.Compose([transforms.ToTensor()])

# Load MNIST training and test datasets.
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# Create data loaders.
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [7]:
ann = ArtificialNeuralNetwork(
    123,
    Layer(
        weight_init=InitializerType.RANDOM_DIST_UNIFORM,
        bias_init=InitializerType.ZERO,
        input_size=input_size,
        num_neurons=hidden_size,
        param_1=param_1,
        param_2=param_2,
        activation=ActivationFunction.relu,
        layer_name=f"Hidden Layer 0"
    ),
    *[Layer(
        weight_init=InitializerType.RANDOM_DIST_UNIFORM,
        bias_init=InitializerType.ZERO,
        input_size=hidden_size,
        num_neurons=hidden_size,
        param_1=param_1,
        param_2=param_2,
        activation=ActivationFunction.relu,
        layer_name=f"Hidden Layer {i + 1}"
    ) for i in range(hidden_layers)],
    OutputLayer(
        weight_init=InitializerType.RANDOM_DIST_UNIFORM,
        bias_init=InitializerType.ZERO,
        input_size=hidden_size,
        num_neurons=output_size,
        param_1=param_1,
        param_2=param_2,
        activation=ActivationFunction.relu,
        layer_name="Output Layer"
    )
)

In [ ]:
ann.train(
    train_loader,
    lr=0.005,
    epochs=200,
    loss_function=LossFunction.categorical_cross_entropy
)

Epoch 1: Loss = 1.7358
Epoch 2: Loss = 1.7277
Epoch 3: Loss = 1.7292
Epoch 4: Loss = 1.7264
Epoch 5: Loss = 1.7239
Epoch 6: Loss = 1.7263
Epoch 7: Loss = 1.7202
Epoch 8: Loss = 1.7225
Epoch 9: Loss = 1.7244
Epoch 10: Loss = 1.7180
Epoch 11: Loss = 1.7184
Epoch 12: Loss = 1.7181
Epoch 13: Loss = 1.7183
Epoch 14: Loss = 1.7156
Epoch 15: Loss = 1.7127
Epoch 16: Loss = 1.7107
Epoch 17: Loss = 1.7068
Epoch 18: Loss = 1.7097
Epoch 19: Loss = 1.7035
Epoch 20: Loss = 1.7039
Epoch 21: Loss = 1.7085
Epoch 22: Loss = 1.7075
Epoch 23: Loss = 1.7040
Epoch 24: Loss = 1.7025
Epoch 25: Loss = 1.6988
Epoch 26: Loss = 1.6994
Epoch 27: Loss = 1.6998
Epoch 28: Loss = 1.6962
Epoch 29: Loss = 1.6976
Epoch 30: Loss = 1.7000
Epoch 31: Loss = 1.6986
Epoch 32: Loss = 1.6944
Epoch 33: Loss = 1.6926
Epoch 34: Loss = 1.6959
Epoch 35: Loss = 1.6873
Epoch 36: Loss = 1.6905
Epoch 37: Loss = 1.6882
Epoch 38: Loss = 1.6865
Epoch 39: Loss = 1.6824
Epoch 40: Loss = 1.6901
Epoch 41: Loss = 1.6854
Epoch 42: Loss = 1.6853
E

In [13]:
ann.test(test_loader)

Test Accuracy: 98.03%
